In [1]:
import pandas as pd
pd.options.display.max_columns = 9999
pd.options.display.max_rows = 999

In [2]:
train_df = pd.read_csv('train.csv', encoding='latin')


In [3]:
test_df = pd.read_csv('test.csv', encoding='latin')

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

/Users/bort/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [5]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
#clean
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
train_df['Age'].mean()

29.69911764705882

In [9]:
#replace NaN age with mean
train_df['Age'] = train_df['Age'].fillna(30)

In [10]:
train_df = train_df.drop(['Cabin'], axis=1)

In [11]:
train_df = train_df.drop(['Ticket'], axis=1)

In [12]:
train_df = train_df.drop(['Name'], axis=1)

In [13]:
def convert_sex(row):
    if row['Sex'] == 'male':
        row['Sex'] = 0
    if row['Sex'] == 'female':
        row['Sex'] = 1
    return row

train_df = train_df.apply(convert_sex, axis='columns')

In [14]:
test = train_df['Embarked'].unique()
test

array(['S', 'C', 'Q', nan], dtype=object)

In [15]:
train_df = train_df[train_df['Embarked'].isin(['S','C','Q'])]
test

array(['S', 'C', 'Q', nan], dtype=object)

In [16]:
#convert embarked
def convert_embarked(row):
    if row['Embarked'] == 'S':
        row['Embarked'] = 0
    if row['Embarked'] == 'C':
        row['Embarked'] = 1
    if row['Embarked'] == 'Q':
        row['Embarked'] = 2
    return row
train_df = train_df.apply(convert_embarked, axis='columns')

In [17]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [18]:
train_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.00,1,0,7.2500,0
1,2,1,1,1,38.00,1,0,71.2833,1
2,3,1,3,1,26.00,0,0,7.9250,0
3,4,1,1,1,35.00,1,0,53.1000,0
4,5,0,3,0,35.00,0,0,8.0500,0
5,6,0,3,0,30.00,0,0,8.4583,2
6,7,0,1,0,54.00,0,0,51.8625,0
7,8,0,3,0,2.00,3,1,21.0750,0
8,9,1,3,1,27.00,0,2,11.1333,0
9,10,1,2,1,14.00,1,0,30.0708,1


In [19]:
y = train_df['Survived']

In [20]:
x = train_df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [21]:
#DecisionTreeRegressor to generate model
model = DecisionTreeRegressor(random_state=1)
model.fit(x, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

In [22]:
predicted = model.predict(x)
mean_absolute_error(y, predicted)

0.024481761208420372

In [23]:
#format test data
test_df = pd.read_csv('test.csv', encoding='latin')

test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())
test_df = test_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
test_df = test_df[test_df['Embarked'].isin(['S','C','Q'])]
def convert_embarked(row):
    if row['Embarked'] == 'S':
        row['Embarked'] = 0
    if row['Embarked'] == 'C':
        row['Embarked'] = 1
    if row['Embarked'] == 'Q':
        row['Embarked'] = 2
    return row
test_df = test_df.apply(convert_embarked, axis='columns')
def convert_sex(row):
    if row['Sex'] == 'male':
        row['Sex'] = 0
    if row['Sex'] == 'female':
        row['Sex'] = 1
    return row
test_df = test_df.apply(convert_sex, axis='columns')
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.50000,0,0,7.829200,2
1,893,3,1,47.00000,1,0,7.000000,0
2,894,2,0,62.00000,0,0,9.687500,2
3,895,3,0,27.00000,0,0,8.662500,0
4,896,3,1,22.00000,1,1,12.287500,0
5,897,3,0,14.00000,0,0,9.225000,0
6,898,3,1,30.00000,0,0,7.629200,2
7,899,2,0,26.00000,1,1,29.000000,0
8,900,3,1,18.00000,0,0,7.229200,1
9,901,3,0,21.00000,2,0,24.150000,0


In [24]:
test_df.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [25]:
tx = test_df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
predicted = model.predict(tx)

In [45]:
test_df1 = test_df
test_df1['Survived'] = predicted

In [46]:
test_df1

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,0,34.50000,0,0,7.829200,2,0.000000
1,893,3,1,47.00000,1,0,7.000000,0,0.000000
2,894,2,0,62.00000,0,0,9.687500,2,1.000000
3,895,3,0,27.00000,0,0,8.662500,0,1.000000
4,896,3,1,22.00000,1,1,12.287500,0,1.000000
5,897,3,0,14.00000,0,0,9.225000,0,0.000000
6,898,3,1,30.00000,0,0,7.629200,2,0.000000
7,899,2,0,26.00000,1,1,29.000000,0,0.000000
8,900,3,1,18.00000,0,0,7.229200,1,1.000000
9,901,3,0,21.00000,2,0,24.150000,0,0.000000


In [47]:
def round_survived(row):
    if row['Survived'] < 0.5:
        row['Survived'] = 0
    else:
        row['Survived'] = 1
    return row
test_df1 = test_df1.apply(round_survived, axis='columns')

In [48]:
test_df1

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892.0,3.0,0.0,34.50000,0.0,0.0,7.829200,2.0,0.0
1,893.0,3.0,1.0,47.00000,1.0,0.0,7.000000,0.0,0.0
2,894.0,2.0,0.0,62.00000,0.0,0.0,9.687500,2.0,1.0
3,895.0,3.0,0.0,27.00000,0.0,0.0,8.662500,0.0,1.0
4,896.0,3.0,1.0,22.00000,1.0,1.0,12.287500,0.0,1.0
5,897.0,3.0,0.0,14.00000,0.0,0.0,9.225000,0.0,0.0
6,898.0,3.0,1.0,30.00000,0.0,0.0,7.629200,2.0,0.0
7,899.0,2.0,0.0,26.00000,1.0,1.0,29.000000,0.0,0.0
8,900.0,3.0,1.0,18.00000,0.0,0.0,7.229200,1.0,1.0
9,901.0,3.0,0.0,21.00000,2.0,0.0,24.150000,0.0,0.0


In [49]:
test_df1 = test_df1.drop(['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'], axis=1)
test_df1

,PassengerId,Survived
0,892.0,0.0
1,893.0,0.0
2,894.0,1.0
3,895.0,1.0
4,896.0,1.0
5,897.0,0.0
6,898.0,0.0
7,899.0,0.0
8,900.0,1.0
9,901.0,0.0


In [56]:
test_df2 = test_df1.astype('int32')

In [57]:
test_df1.dtypes

PassengerId    float64
Survived       float64
dtype: object

In [58]:
test_df2.dtypes

PassengerId    int32
Survived       int32
dtype: object

In [59]:
test_df2

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [60]:
test_df2.to_csv('bort_submission.csv',index=False)